# Hadoop Streaming assignment 2: Stop Words

In [1]:
%%writefile mapper.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

path = 'stop_words_en.txt'

with open(path) as f:
    stop_words = set(f.read().split())

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        print >> sys.stderr, "reporter:counter:wiki,total_words,%d" % 1
        if word in stop_words:
            print >> sys.stderr, "reporter:counter:wiki,stop_words,%d" % 1
        print "%s\t%d" % (word.lower(), 1)

Overwriting mapper.py


In [2]:
%%writefile counter_process.py

import sys
import re

output_log = list(map(lambda x: x.strip(), sys.stdin.read().split()))

pattern_tot = 'total_words='
regexp_tot = re.compile(pattern_tot)

pattern_stop = 'stop_words='
regexp_stop = re.compile(pattern_stop)

total_words = [int(x.replace(pattern_tot, '')) for x in output_log if regexp_tot.search(x)][0]
stop_words = [int(x.replace(pattern_stop, '')) for x in output_log if regexp_stop.search(x)][0]

print(stop_words / float(total_words) * 100)

Overwriting counter_process.py


In [3]:
%%bash

OUT_DIR="coursera_mr_task_stop_words"
NUM_REDUCERS=0
LOGS="stderr_logs.txt"

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming stopwords" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,/datasets/stop_words_en.txt \
    -mapper "python mapper.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null 2> ${LOGS}

cat ${LOGS} | egrep "*_words" | python ./counter_process.py #"Stop words" "Total words"
cat ${LOGS} >&2


38.44036900909957


19/02/10 02:28:32 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/02/10 02:28:32 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/02/10 02:28:33 INFO mapred.FileInputFormat: Total input files to process : 1
19/02/10 02:28:33 INFO mapreduce.JobSubmitter: number of splits:2
19/02/10 02:28:34 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1549762393391_0005
19/02/10 02:28:34 INFO impl.YarnClientImpl: Submitted application application_1549762393391_0005
19/02/10 02:28:34 INFO mapreduce.Job: The url to track the job: http://fef6b3b3df29:8088/proxy/application_1549762393391_0005/
19/02/10 02:28:34 INFO mapreduce.Job: Running job: job_1549762393391_0005
19/02/10 02:28:40 INFO mapreduce.Job: Job job_1549762393391_0005 running in uber mode : false
19/02/10 02:28:40 INFO mapreduce.Job:  map 0% reduce 0%
19/02/10 02:28:58 INFO mapreduce.Job:  map 23% reduce 0%
19/02/10 02:29:04 INFO mapreduce.Job:  map 35% reduce 0%
19/02/10 02:29:10 INFO 